# Import related functions

In [2]:
import sys
import cobra
import pandas as pd
import re
sys.path.append(r'./script/')
from ECMpy_function import *
from AutoPACMEN_function import *

# Input and output files

In [2]:
# Initial parameter
obj='Biomass'
fluxes_outfile = './analysis/ECMpy_solution_%s_pfba.csv'%obj 
use_substrate='R2399'# substrate ID
concentration=10 #
# The total protein fraction in cell.

ptot = 0.4653
# The approximated average saturation of enzyme.

sigma = 0.5
# Lowerbound  of enzyme concentration constraint. 

lowerbound = 0   
enz_ratio=0.1

# Input files
autopacmen_folder = "./iDL1450_get_data/"
gap_fill= 'mean'#'mean'#'median'
project_name = "iDL1450%s"%gap_fill
organism = "Thermothelomyces thermophilus"
create_file(autopacmen_folder)
reaction_gene_path = "./data/reaction_gene_subunit_num.json"

protein_kcat_database_path = "none"
bigg_metabolites_file = "./data/bigg_models_metabolites.txt"#date:20221008 http://bigg.ucsd.edu/static/namespace/bigg_models_metabolites.txt
brenda_textfile_path = "./data/brenda_download.txt"#date:20221008 https://www.brenda-enzymes.org/brenda_download/file_download.php

sbml_path="./model/iYW1475.xml"

c13reaction_file='./reaction_kcat/C13data.csv'#C13 data
file_list=[gene_abundance_file1,gene_abundance_file2,gene_abundance_file3,gene_abundance_file4]
EC_max_file='./data/EC_kcat_max.json'
brenda_json_path = "%skcat_database_brenda.json"%autopacmen_folder
brenda_output_json_path = "%skcat_database_brenda_for_model.json"%autopacmen_folder
bigg_id_name_mapping_path = "%sbigg_id_name_mapping.json"%autopacmen_folder
sabio_rk_output_path = "%skcat_database_sabio_rk.json"%autopacmen_folder
combined_output_path = "%skcat_database_combined.json"%autopacmen_folder
reaction_kcat_MW_file="%s/reaction_kcat_MW.csv"%autopacmen_folder
uniprot_data_file='%s/%s_protein_id_mass_mapping.json'%(autopacmen_folder,project_name)


modelgene2uniprot_file='./analysis/modelgene2uniprot.txt'
paxdbgene2uniprot_file='./analysis/paxdbgene2uniprot.txt'
ecModel_output_file="./model/ecMTM_TurNup24_1.json" 

json_output_file = './model/ecMTM_TurNup_adj_enz24_1.json'
reaction_kcat_MW_outfile="./get_reaction_kcat/kcat/reaction_change_by_enzuse_TurNup24_1.csv"
reaction_kcat_MW_round2_outfile='./get_reaction_kcat/kcat/reaction_change_by_C13_TurNup24_1.csv'


path exists


### Calculate f-value

In [3]:
model=cobra.io.read_sbml_model(sbml_path)
# Proteome data
gene_abundance_file1 = './data/danbaizu/wt1.csv'
gene_abundance_file2 = './data/danbaizu/wt2.csv'
gene_abundance_file3 = './data/danbaizu/wt3.csv'
gene_abundance_file4 = './data/danbaizu/wt4.csv'
file_list=[gene_abundance_file1,gene_abundance_file2,gene_abundance_file3,gene_abundance_file4]
f_list=[]
for g_a_file in file_list:
    f=calculate_f_special_new(model,g_a_file)
    f_list.append(f)
f_list

[0.5563635421646973, 0.5540772163224917, 0.558660486869679, 0.5486668910172237]

# Constructing Enzyme-Constrained Mode

In [4]:
#The enzyme mass fraction 
f = round(np.mean(f_list), 2)
# print("f is:" + str(f))
ptot = 0.4653

upperbound = round(ptot * f * sigma, 3)
# upperbound = 0.1
print("upperbound is:" + str(upperbound))

reaction_kcat_MW_file="./get_reaction_kcat/kcat/TurNup.csv"# kcat data predicted by TurNup method

trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, ecModel_output_file)

enz_model=get_enzyme_constraint_model(ecModel_output_file)
enz_model.objective=obj
enz_model.summary()
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-10, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_output_file)
print(enz_model_pfba_solution.fluxes[obj])

upperbound is:0.128
0.128
IN FLUXES         OUT FLUXES     OBJECTIVES
----------------  -------------  --------------
glc__D_e  5.56    co2_e   10.5   Biomass  0.083
h_e       5.14    etoh_e   9.78
nh4_e     0.621   h2o_e    2.22
o2_e      0.563
pi_e      0.0151
so4_e     0.0012
Enzyme cost total is:0.128
0.08297404584306473


# Calibration kcat according Enzyme usage

In [5]:

#Initial model and result
fluxes_infile_ori = './analysis/ECMpy_solution_%s_pfba.csv'%obj
fluxes_outfile = fluxes_infile_ori
need_change_reaction_list=[]
changed_reaction_list=[]
round_num=1


reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#Model adjustment
enz_model_obj_max=0
concentration = 3.0676
# Reconstruction and analysis of genome-scale metabolic model for thermophilic fungus Myceliophthora thermophila [J]. Biotechnology and Bioengineering, 2022, 119(7): 1926-37.
# Beyond growth rate 0.25: Elucidation of the metabolic mechanism for malate production in Myceliophthora thermophila via 13C metabolic flux analysis [Z]. Research Square. 2022.10.21203/rs.3.rs-2123109/v1
while (enz_model_obj_max<0.2573):

 
 
    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(enz_ratio,\
                                                    ecModel_output_file,fluxes_outfile,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
   
    #Using only the provided substrate
  
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
    enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)
  
  
    
    enz_model_obj=enz_model.slim_optimize()

    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    fluxes_outfile = './analysis/ECMpy_solution_adjround%s.csv'%(round_num)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_output_file)
    
    

   # μmax condition substrate concentration
  
    obj='Biomass'
    substrate='R2399'
    enz_model.reactions.get_by_id(substrate).bounds=(-1000,0)
    enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,0)
    
    enz_model.objective=obj
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    print("ecGEM max glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
    print("ecGEM max growth rates:"+str(enz_model_solution.fluxes[obj]))
    enz_model_sub=-enz_model_solution.fluxes[substrate]
    enz_model_obj_max=enz_model_solution.fluxes[obj]


    print('Round %s: '%round_num+str(enz_model_obj))
    round_num=round_num+1
    if round_num>10:
        break
    print()
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)

Need changing reaction: 
R1036
Changed reaction: 
['R1036']
0.128
Enzyme cost total is:0.12799999999999997
Enzyme cost total is:0.12799999999999992
ecGEM max glucose uptake rates:10.89155237955375
ecGEM max growth rates:0.17917603317114514
Round 1: 0.07664756140109978

Need changing reaction: 
R2160
Changed reaction: 
['R1036', 'R2160']
0.128
Enzyme cost total is:0.12799999999999992
Enzyme cost total is:0.1280000000000001
ecGEM max glucose uptake rates:10.749192538227003
ecGEM max growth rates:0.18591312025932422
Round 2: 0.1738360670787642

Need changing reaction: 
R2158
Changed reaction: 
['R1036', 'R2160']
0.128
Enzyme cost total is:0.1280000000000001
Enzyme cost total is:0.1280000000000001
ecGEM max glucose uptake rates:10.749192538227003
ecGEM max growth rates:0.18591312025932422
Round 3: 0.1738360670787642

Need changing reaction: 
R2157
Changed reaction: 
['R1036', 'R2160', 'R2157']
0.128
Enzyme cost total is:0.1280000000000001
Enzyme cost total is:0.12800000000000006
ecGEM max 

# Calibration kcat according 13C

In [6]:

c13_percentage=0.1
# kcat_database_combined_file='./data/EC_kcat_max.json'##max kcat for EC number selected from BRENDA and SABIO-RK database(use autoPACMEN)
model_name=sbml_path#初始的xml文件
f=0.55 # Enzyme mass fraction
ptot=0.4653 # total protein content in cells
sigma=0.5 #approximate average saturation of the enzyme
lowerbound= 0 
upperbound=round(ptot * f * sigma, 3)


json_round2_output_file='./model/ecMTM.json'#Adjusting kcat based on C13 and outputting the adjusted model


enz_model=get_enz_model_use_c13(reaction_kcat_MW_outfile, ecModel_output_file, c13reaction_file, c13_percentage, \
                                EC_max_file,model_name, f, ptot, sigma, lowerbound, \
                                upperbound, json_round2_output_file,reaction_kcat_MW_round2_outfile)
enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,3.06)
enz_model.summary()

0.128
通量为： -1.60059142458337e-11
R6_num1 ECMpy_c13_reaction_flux: 0.009956481984000001 flux: 3.0676
0.128
通量为： -1.60059142458337e-11
R7 ECMpy_c13_reaction_flux: 0.113010559296 flux: 1.4289
0.128
通量为： 5.319665982439708
R1036 ECMpy_c13_reaction_flux: 0.06682355402351355 flux: 1.936
0.128
通量为： 0.0
R1044_num1 ECMpy_c13_reaction_flux: 0.0025007938508800004 flux: 1.936
0.128
通量为： 10.66067912683804
R50 ECMpy_c13_reaction_flux: 0.158462757914232 flux: 4.1708
0.128
通量为： 10.66067912683804
R51 ECMpy_c13_reaction_flux: 0.09369485888000001 flux: 4.1708
0.128
通量为： 0.0
R42_num2 ECMpy_c13_reaction_flux: 0.016192371078400002 flux: 3.9965
0.128
通量为： 10.559987752468738
R40 ECMpy_c13_reaction_flux: 0.011152988908800003 flux: 3.9965
0.128
通量为： 10.402400111536174
R36_num2 ECMpy_c13_reaction_flux: 0.039763349945600004 flux: 3.8319
0.128
通量为： 0.13123741782926515
R55 ECMpy_c13_reaction_flux: 0.059693464153599995 flux: 1.2271
0.128
通量为： 0.13123741782926515
R60 ECMpy_c13_reaction_flux: 0.012909565510400001 flux:

# Obtain the metabolic flux of this model

In [4]:
ecModel_output_file='./model/ecMTM.json'
enz_model=get_enzyme_constraint_model(ecModel_output_file)
enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,3.06)
enz_model.reactions.get_by_id("R1975_num1").bounds=(2.8,2.8)
enz_model.reactions.get_by_id("R1975_num2").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num3").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num4").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num5").bounds=(0,0)
solution = enz_model.optimize()
reac=dict(solution.fluxes[solution.fluxes> 0.1])
genelist=[]
reactionlist=[]
fluxlist=[]
for i in reac:
    reID=str(enz_model.reactions.get_by_id(i))
    reID=str(enz_model.reactions.get_by_id(i)).split(':')[0]
    reID= reID.split('_')[0]
    reactionlist.append(reID)
    fluxlist.append(reac[i])
df=pd.DataFrame({'reaction':reactionlist,'flux':fluxlist})

df.to_csv("./flux/flux for TN.csv",index=0)


0.128
